<a href="https://colab.research.google.com/github/sbooeshaghi/BMGP_2020/blob/main/analysis/notebooks/clicktag/clicktag_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!date

Mon Jun 14 22:21:46 UTC 2021


# Install packages and download data

In [2]:
import sys
COLAB = "google.colab" in sys.modules


In [3]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 490 (delta 61), reused 124 (delta 28), pack-reused 296
Receiving objects: 100% (490/490), 425.96 MiB | 27.41 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Checking out files: 100% (130/130), done.


In [4]:
!pip --quiet install CITE-seq-Count==1.4.4

     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 81kB 5.7MB/s 
     |████████████████████████████████| 225kB 40.2MB/s 
     |████████████████████████████████| 9.9MB 19.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6 \
    CITE-seq-Count==1.4.4

     |████████████████████████████████| 59.1MB 73kB/s 
     |████████████████████████████████| 133kB 45.2MB/s 
     |████████████████████████████████| 10.3MB 25.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 13.2MB 144kB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 112kB 55.5MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 1.2MB 30.7MB/s 


In [6]:
if COLAB:
    # download the relevant data
    !wget -O clicktag_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/zqaom7yuul7ujetqyhnd4lvf8vhzqsyg.gz \
    2>&1
    !tar -xf clicktag_fastqs.tar.gz

clicktag_fastqs.tar 100%[===================>]   1.47G  24.7MB/s    in 79s     


# Preprocess with kb

In [7]:
%%time
!INDEX="BMGP_2020/references/clicktag/kite/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite/features.fa" && \
FB="BMGP_2020/references/clicktag/kite/feature_barcodes.txt" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-06-14 22:24:43,832]    INFO Generating mismatch FASTA at BMGP_2020/references/clicktag/kite/features.fa
[2021-06-14 22:24:43,844]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/clicktag/kite/f2b.txt
[2021-06-14 22:24:43,848]    INFO Skipping kallisto index because BMGP_2020/references/clicktag/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 15 ms, sys: 5.59 ms, total: 20.5 ms
Wall time: 1.62 s


## No mismatches comparison

In [12]:
%%time
#FB="BMGP_2020/references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
!FB="BMGP_2020/references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/clicktag/kite_no_mismatches/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite_no_mismatches/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite_no_mismatches/features.fa" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
--no-mismatches \
$FB

[2021-06-14 22:30:55,157]    INFO Generating mismatch FASTA at BMGP_2020/references/clicktag/kite_no_mismatches/features.fa
[2021-06-14 22:30:55,171]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/clicktag/kite_no_mismatches/f2b.txt
[2021-06-14 22:30:55,172]    INFO Skipping kallisto index because BMGP_2020/references/clicktag/kite_no_mismatches/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 12.9 ms, sys: 6.97 ms, total: 19.9 ms
Wall time: 1.11 s


In [8]:
%%bash
FB="BMGP_2020/references/clicktag/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/clicktag/kite/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-14 22:24:47,297]    INFO Using index BMGP_2020/references/clicktag/kite/features.idx to generate BUS file to kite_out from
[2021-06-14 22:24:47,297]    INFO         fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz
[2021-06-14 22:24:47,297]    INFO         fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz
[2021-06-14 22:26:39,862]    INFO Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-06-14 22:26:52,738]    INFO Whitelist not provided
[2021-06-14 22:26:52,739]    INFO Copying pre-packaged 10XV2 whitelist to kite_out
[2021-06-14 22:26:52,849]    INFO Inspecting BUS file kite_out/tmp/output.s.bus
[2021-06-14 22:26:58,524]    INFO Correcting BUS records in kite_out/tmp/output.s.bus to kite_out/tmp/output.s.c.bus with whitelist kite_out/10xv2_whitelist.txt
[2021-06-14 22:27:04,473]    INFO Sorting BUS file kite_out/tmp/output.s.c.bus to kite_out/output.unfiltered.bus
[2021-06-14 22:27:16,085]    INFO Generating count matrix kite_out/counts_unfiltered/cells_x_features from B

In [9]:
!cat kite_out/run_info.json

{
	"n_targets": 0,
	"n_bootstraps": 0,
	"n_processed": 15928888,
	"n_pseudoaligned": 15357069,
	"n_unique": 15357069,
	"p_pseudoaligned": 96.4,
	"p_unique": 96.4,
	"kallisto_version": "0.46.2",
	"index_version": 0,
	"start_time": "Mon Jun 14 22:24:47 2021",
	"call": "/usr/local/lib/python3.7/dist-packages/kb_python/bins/linux/kallisto/kallisto bus -i BMGP_2020/references/clicktag/kite/features.idx -o kite_out -x 10xv2 -t 8 fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz"
}


In [14]:
%%bash
FB="BMGP_2020/references/clicktag/kite_no_mismatches/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/clicktag/kite_no_mismatches/features.idx" && \
F2B="BMGP_2020/references/clicktag/kite_no_mismatches/f2b.txt" && \
FASTA="BMGP_2020/references/clicktag/kite_no_mismatches/features.fa" && \
OUT="kite_no_mismatches" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv2 \
-o $OUT \
--workflow kite \
--h5ad \
--filter bustools \
--overwrite \
$FASTQS

[2021-06-14 22:31:24,101]    INFO Using index BMGP_2020/references/clicktag/kite_no_mismatches/features.idx to generate BUS file to kite_no_mismatches from
[2021-06-14 22:31:24,101]    INFO         fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz
[2021-06-14 22:31:24,101]    INFO         fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz
[2021-06-14 22:33:01,777]    INFO Sorting BUS file kite_no_mismatches/output.bus to kite_no_mismatches/tmp/output.s.bus
[2021-06-14 22:33:14,742]    INFO Whitelist not provided
[2021-06-14 22:33:14,742]    INFO Copying pre-packaged 10XV2 whitelist to kite_no_mismatches
[2021-06-14 22:33:14,849]    INFO Inspecting BUS file kite_no_mismatches/tmp/output.s.bus
[2021-06-14 22:33:20,465]    INFO Correcting BUS records in kite_no_mismatches/tmp/output.s.bus to kite_no_mismatches/tmp/output.s.c.bus with whitelist kite_no_mismatches/10xv2_whitelist.txt
[2021-06-14 22:33:26,175]    INFO Sorting BUS file kite_no_mismatches/tmp/output.s.c.bus to kite_no_mismatches/output.unfiltere

In [16]:
!cat kite_no_mismatches/run_info.json

{
	"n_targets": 0,
	"n_bootstraps": 0,
	"n_processed": 15928888,
	"n_pseudoaligned": 15058183,
	"n_unique": 15058183,
	"p_pseudoaligned": 94.5,
	"p_unique": 94.5,
	"kallisto_version": "0.46.2",
	"index_version": 0,
	"start_time": "Mon Jun 14 22:31:24 2021",
	"call": "/usr/local/lib/python3.7/dist-packages/kb_python/bins/linux/kallisto/kallisto bus -i BMGP_2020/references/clicktag/kite_no_mismatches/features.idx -o kite_no_mismatches -x 10xv2 -t 8 fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz"
}


# Preprocess with cite-seq-count

In [ ]:
!gunzip BMGP_2020/references/whitelist_10xv2.txt.gz

In [ ]:
!R1="fastqs/NSC_Fixed_S2_L001_R1_001.fastq.gz" && \
R2="fastqs/NSC_Fixed_S2_L001_R2_001.fastq.gz" && \
FB="BMGP_2020/references/clicktag/cite_seq_barcodes.csv" && \
WL="BMGP_2020/references/whitelist_10xv2.txt" && \
/usr/bin/time -v CITE-seq-Count -R1 $R1  -R2 $R2 --no_umi_correction -t $FB -wl $WL -cbf 1 -cbl 16 -umif 17 -umil 26 -cells 21000 --start-trim 28  -o ./cite_seq_out

Loading whitelist
Counting number of reads
Started mapping
Processing 15,928,888 reads
CITE-seq-Count is running with 48 cores.
Mapping done for process 48987. Processed 331,851 reads
Mapping done for process 48988. Processed 331,851 reads
Mapping done for process 48990. Processed 331,851 reads
Mapping done for process 48989. Processed 331,851 reads
Mapping done for process 48991. Processed 331,851 reads
Mapping done for process 48992. Processed 331,851 reads
Mapping done for process 48993. Processed 331,851 reads
Mapping done for process 48994. Processed 331,851 reads
Mapping done for process 48995. Processed 331,851 reads
Mapping done for process 48996. Processed 331,851 reads
Mapping done for process 48997. Processed 331,851 reads
Mapping done for process 48998. Processed 331,851 reads
Mapping done for process 48999. Processed 331,851 reads
Mapping done for process 49000. Processed 331,851 reads
Mapping done for process 49001. Processed 331,851 reads
Mapping done for process 49002. 